In [1]:
import io,os,sys,json
from types import SimpleNamespace
from collections import OrderedDict

notebook_dir = os.getcwd()
print('notebook dir: ', notebook_dir)
project_dir = os.path.dirname(notebook_dir)
print('project dir: ', project_dir)
sys.path.insert(0, project_dir)
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "dingdong.settings")

import django
django.setup()

def print_qs(querySet):
    from django.core import serializers
    data = serializers.serialize("json", querySet, indent=4)
    print(data)

notebook dir:  D:\workspace\python\dingdong\notebooks
project dir:  D:\workspace\python\dingdong


In [2]:
from django.contrib.auth import authenticate
from django.contrib.contenttypes.models import ContentType

In [3]:
from rest_framework.request import Request
from rest_framework.test import APIRequestFactory

request_factory = APIRequestFactory()

serializer_context = {
    'request': Request(request_factory.get('/')),
}

from rest_framework.test import RequestsClient
from rest_framework import serializers
from rest_framework.renderers import JSONRenderer

In [4]:
from base.models import Account
from base.api.serializers import AccountSerializer
from rest_framework.decorators import action
from rest_framework import viewsets
from rest_framework.response import Response

In [5]:
auth_data = {'username':'admin', 'password':'123'}
# auth_data = {'username':'ada', 'password':'123'}
##############################################################################################
# authencate
client = RequestsClient()
response = client.post('http://testserver/api/token/obtain/', auth_data)
assert response.status_code == 200
token = json.loads(response.content.decode("utf-8"))
client.headers.update({'Authorization': 'Bearer ' + token['access']})

def test(method, url, payload=None):
    if method == 'get':
        response = client.get(url)
    elif method == 'post':
        response = client.post(url, payload)
    elif method == 'put':
        response = client.put(url, payload)
    elif method == 'patch':
        response = client.patch(url, payload)
    elif method == 'delete':
        response = client.delete(url)
#     assert response.status_code == 200
    print('response status code:', response.status_code)
    result = json.loads(response.content.decode("utf-8"))
    print(json.dumps(result, indent=4))
###########################################################################################

In [6]:
url = 'http://testserver/api/base/accounts/'
url = 'http://testserver/api/base/accounts/2/'
url = 'http://testserver/api/base/accounts/current_account'

In [6]:
payload = {
    'first_name': 'ada',
    'last_name': 'alva'
}

In [8]:
url = 'http://testserver/api/base/accounts/1/set_password/'
test('patch', url, payload)

1 1
response status code: 200
{
    "status": "password set"
}


In [7]:
test('get', url)

response status code: 200
{
    "account": {
        "url": "http://testserver/api/base/accounts/1/",
        "id": 1,
        "last_login": "2019-04-18T13:05:10.407877Z",
        "is_superuser": true,
        "username": "admin",
        "first_name": "",
        "last_name": "",
        "email": "odingdongo@hotmail.com",
        "is_staff": true,
        "is_active": true,
        "date_joined": "2019-03-27T13:18:56.634763Z"
    },
    "permissions": []
}


SyntaxError: can't assign to literal (<ipython-input-14-316961992ad3>, line 1)

In [13]:
b

'cd'